In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
ds = pd.read_csv('./mushrooms.csv')
print ds.shape
ds.head()

(8124, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
## Map from alphabet space to integer space
le = LabelEncoder()
df = ds.apply(le.fit_transform)
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [19]:
## Splitting training and testing dataset
split = int(df.shape[0]*0.9)
print split
train = df.loc[:split]
print type(train)
test = df.loc[split:]
print type(test)
print train.shape
print test.shape
test.head()
test = test.reset_index(drop=True)
test.head()

7311
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(7312, 23)
(813, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,5,0,8,0,5,1,1,0,2,...,2,7,7,0,2,2,4,7,3,1
1,1,3,3,2,0,8,1,0,1,0,...,2,7,6,0,2,1,0,7,4,2
2,1,3,3,4,0,7,1,0,1,0,...,1,7,7,0,2,1,0,7,4,4
3,0,5,2,3,0,5,1,1,0,7,...,2,7,7,0,2,2,4,7,2,1
4,0,5,2,8,0,5,1,1,0,2,...,1,7,7,0,2,2,4,7,2,1


In [28]:
## Functions Required
def prior_prob(data, label_column, label_value):
    n_rows = data.loc[data[label_column]==label_value]
    prob_val = n_rows.shape[0]/float(data.shape[0])
    return prob_val
    
print prior_prob(df, 'type', 1)
    
def cond_prob(data, feature_column, feature_value, label_column, label_value):
    
    n_rows = data.loc[data[label_column]==label_value]
    constraint_rows = data.loc[(data[label_column]==label_value) & (data[feature_column]==feature_value)]
    
    prob_val = constraint_rows.shape[0]/float(n_rows.shape[0])
    return prob_val

0.482028557361


In [27]:
temp = df.values[:,0]
print temp.shape
x = np.unique(temp, return_counts=True)
print x
count = 0
for ix in x[1]:
    print "Type ", x[0][count], " ", ix/float(x[1].sum())
    count+=1

(8124,)
(array([0, 1]), array([4208, 3916]))
Type  0   0.517971442639094
Type  1   0.48202855736090594


In [32]:
## Naive Bayes algorithm implementation
CLASSES = np.unique(df['type'])
print CLASSES
preds = []
ctr = 0
for ix in range(test.shape[0]):
    if ctr%100==0:
        print ctr
    
    ctr+=1
    
    val = test.loc[ix][1:]
    features = dict(val)
    
    posterior_probs = []
    for cx in CLASSES:
        likelihood = 1.0
        for fx in features:
            cp = cond_prob(train, fx, features[fx], 'type', cx)
            likelihood *= cp
            
        prior = prior_prob(train, 'type', cx)
        post_prob = prior*likelihood
        posterior_probs.append(post_prob)
    
    posterior_probs = np.array(posterior_probs)
    pred_label = posterior_probs.argmax()
    preds.append(pred_label) ## Prediction for the current data point

[0 1]
0
100
200
300
400
500
600
700
800


In [36]:
true_labels = np.array(test['type'])
print true_labels.shape

pred_labels = np.array(preds)
print pred_labels.shape

accuracy = np.sum(np.array(true_labels==pred_labels))/float(true_labels.shape[0])
print accuracy

(813,)
(813,)
0.97539975399754


In [40]:
temp = test.loc[0][1:]
print temp.shape
f = dict(temp)
print f

(22,)
5
{'stalk_root': 0, 'stalk_color_above_ring': 7, 'stalk_surface_above_ring': 2, 'gill_attachment': 1, 'gill_spacing': 1, 'habitat': 1, 'stalk_shape': 0, 'spore_print_color': 7, 'gill_size': 0, 'gill_color': 2, 'ring_number': 2, 'veil_type': 0, 'odor': 5, 'stalk_color_below_ring': 7, 'cap_surface': 0, 'ring_type': 4, 'population': 3, 'veil_color': 2, 'stalk_surface_below_ring': 2, 'bruises': 0, 'cap_color': 8, 'cap_shape': 5}


In [31]:
l = np.array([1,2,4,3,1])
l.argmax()

2

In [38]:
'''Homework Assignment'''
def get_prediction_and_confidence(test_point):
    '''Code Goes here'''
    pass
    #return pred, conf